In [ ]:
import torch
import os
from model_training import MultiTaskColorModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_folder = 'models'
model_name = '100_0.1_1.0_color_model.pth'
model_path = os.path.join(model_folder, model_name)
# model_path = 'Normalized_100_1.0_1.0_color_model.pth'
# model_name = '100_0.1_1.0_color_model.pth'
# model_path = '50_1.0_1.0_color_model.pth'
# model_path = '50_0.0_1.0_color_model.pth'
# model_path = 'ResNet_152_50_0.1_1.0.pth'
# model_path = 'ResNet_152_50_0.1_1.0_v2.pth';
model = MultiTaskColorModel(num_classes=8, regression_output_size=4)
# model = torch.load('100_0.1_color_model.pth', weights_only=False)
model.load_state_dict(torch.load(model_path, weights_only=True))
model = model.to(device)

model.eval()
total_correct = 0
total_samples = 0
total_reg_loss = 0.0


C:\Users\zjr02\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\zjr02\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
from model_training import ColorDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn

csv_file = r'D:\COSI149\color_with_label\label.csv'
img_dir = r'D:\COSI149\color_with_label'
original_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # if trainning data was normalized, then test data should also be normalized
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ColorDataset(csv_file=csv_file, img_dir=img_dir, transform=original_transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


criterion_class = nn.CrossEntropyLoss()
criterion_reg = nn.MSELoss()

with torch.no_grad():
    for images, labels, color_codes, encoded_labels in dataloader:
        images = images.to(device)
        labels = encoded_labels.to(device)
        color_codes = color_codes.to(device)

        class_outputs, reg_outputs = model(images)

        # Classifacation accuracy
        _, predicted = torch.max(class_outputs, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

        # Regression loss
        loss_reg = criterion_reg(reg_outputs, color_codes)
        total_reg_loss += loss_reg.item()

    accuracy = total_correct / total_samples
    avg_reg_loss = total_reg_loss / len(dataloader)
    print(f"Validation Accuracy: {accuracy*100:.2f}%")
    print(f"Average Regression Loss: {avg_reg_loss:.4f}")


# record the model's performance with timestamp
import datetime
with open('model_performance.txt', 'a') as f:
    f.write(f"{datetime.datetime.now()}\n")
    f.write(f"Model: {model_path}\n")
    f.write(f"Validation Accuracy: {accuracy*100:.2f}%\n\n")
    

Validation Accuracy: 81.25%
Average Regression Loss: 10.2882
